In [ ]:
#from langchain.llms import Ollama
from langchain.chains import LLMChain
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from langgraph.types import interrupt
from langgraph.checkpoint.memory import MemorySaver
from operator import add
from langgraph.types import Command
from langchain_ollama import OllamaLLM

In [ ]:
llm = OllamaLLM(base_urk="http://localhost:11434", model = "deepseek-r1:32b", streaming=True)

In [ ]:

#llm1 = OllamaLLM(base_urk="http://localhost:11434", model = "llama3.2:1b", streaming=True)

In [ ]:
class State(TypedDict):
    bugAnalized: str
    bugDescription: str
    historicalData: str
    reproduceSuggestion: str
    testResult: str
    reviewResult: str
    failReproduce: str
    userReview: str


In [ ]:
def printSteam(llm, prompt="Hello"):
    output = ""
    for chunk in llm.stream(prompt):
        print(chunk, end="", flush=True)
        output += chunk
    #print(output)
    return output

In [ ]:
def getRelatedBugs(description):
    related =  '''example 1:
"""<example>
#ID: 26967 

#bin:
ibcf 7.2.52

#config:
[PEER_valid_auto_ext]
SIP_MANIPULATION[] = SET_ALLOW
Authorization_profile = 0
Max_in = 65550
Max_out = 65550
Hiding = NO_HIDING
Media_Profile = 19
Routing_profile = 0
behavior = 7
Trusted = TRUE
Privacy = FALSE
Fields_Filtering = NONE
Domain[] = 192.168.33.33:5022

[ MANIPULATION_RULE_DEL-ALLOW-INSTANCE1 ]
Instance = "^"
Action_Type = Del
Element_Type = HEADER-VALUE
Header_Name = Allow

[ SIP_MANIPULATION_PROFILE_SET_ALLOW ]
Manipulation[] = DEL-ALLOW-INSTANCE1
Applicability = BOTH
Method_Type[] = "*"
Message_Type = BOTH
Strategy = IN

#Message:
(11:15:05.88) W0 in <192.168.33.157:5681> <- 809 bytes from <192.168.33.163:5022>
INVITE sip:0167000000@192.168.33.157:5681 SIP/2.0
To: Called <sip:0167000000@192.168.33.157>
From: Calling <sip:0166000000@192.168.33.163>;tag=1-1-1
Via: SIP/2.0/UDP 192.168.33.163:5022;branch=z9hG4bK-13293-1-1
Call-ID: 1-13293@192.168.33.163
CSeq: 1 INVITE
Contact: <sip:192.168.33.163:5022>
P-Asserted-Identity: <sip:0166000000@192.168.33.163:5022;user=phone>
Route: <sip:192.168.33.157:5681;lr>
Max-Forwards: 70
Allow: REFER,INVITE,INFO,ACK,SUBSCRIBE,UPDATE,REGISTER,BYE,CANCEL
Content-Type: application/sdp
Content-Length:   261

v=0
o=cp10 116221159107 116221159108 IN IP4 192.168.33.163
s=SIP Call
c=IN IP4 192.168.33.163
t=0 0
m=audio 6350 RTP/AVP 0 18 8
b=AS:64
a=rtpmap:0 PCMU/8000/1
a=rtpmap:8 PCMA/8000/1
a=rtpmap:18 G729/8000/1
a=fmtp:18 annexb=no
a=ptime:20
a=sendrecv

#ERROR:
Allow: BYE,CANCEL,INFO,INVITE,REFER,REGISTER,SUBSCRIBE,UPDATE


#Step reproduce:
Step 1: Get the ibcf binary version 7.2.52
Step 2: Get the basic call sipp scenarial
Step 3: Modify header initial INVITE the same as provided INVITE message
Step 4: Config HMR to delete first field in Allow header
Step 5: Make the basic call between ibcf
Step 6: Checking out going INVITE allow header is deleted not the first 'allow' value
</example>"""
example 2:
"""<example>
#ID:
24628 

#bin:
ibcf 7.2.60

#Config:
[ MATCHING_RULE_HAS_TO_TAG ]
Header_Name = TO
Element_Type = HEADER-PARAM, tag
Condition_Type = NONE
Instance = "*"
Min_Occurrence = 1

#Message:
(08:14:09.41) W0 in <10.208.189.17:5060> <- 3991 bytes from <10.208.189.14:5060>
INVITE sip:+49D0781749490177@sipreg2.ewetel.de;user=phone SIP/2.0
Call-ID: fdbb820566c142ebe980de659c7ec103@10.208.189.40
Contact: <sip:+4944135012780@10.208.189.40:5060>
Content-Type: application/sdp
CSeq: 7 INVITE
From: <sip:+4944135012780@sipreg2.voice.ewetel.de;user=phone>;tag=33038721_09ba9498_2bc2794a-788a-4b33-bf00-64d448102706
Max-Forwards: 63
Record-Route: <sip:10.208.189.14:5060;user=i0o0S000005b7;lr;Cpkt=CNMKV;C=on-cirpack>
Route: <sip:10.208.189.17:5060;transport=udp;lr>,<sip:ims1p1.nica.ewetel.de;transport=udp;lr>
Supported: 100rel,replaces
To: <sip:+4944180002461@nplcrnat.osp.voice.ewetel.de>
Via: SIP/2.0/UDP 10.208.189.14:5060;branch=z9hG4bK-CNMK-000010d4-198d2a4d,SIP/2.0/UDP 10.208.189.40:5060;received=10.208.189.40;rport=5060;branch=z9hG4bK2bc2794a-788a-4b33-bf00-64d448102706_09ba9498_7827518534571509
Accept: application/sdp,multipart/mixed
Accept-Encoding: identity
Allow: INVITE,BYE,ACK,OPTIONS,CANCEL,INFO,PRACK,SUBSCRIBE,NOTIFY,REFER,UPDATE,MESSAGE,PUBLISH
Allow-Events: telephone-event,refer
Expires: 120
User-Agent: AVM FRITZ!Box 7590 154.07.29 (Oct 26 2021)
Session-ID: 05ef8fc121ed9814c6ad31e4cbe74262
P-Asserted-Identity: <sip:+4944135012780@sipreg2.voice.ewetel.de;user=phone>
History-info: <sip:+4944180007922@icscf-cl02.ims1.voice.ewetel.de?Privacy=history>;index=1
History-info: <sip:+49800376462401@sipreg3.voice.ewetel.de;user=phone?Privacy=history&Reason=SIP%3Bcause%3D302%3Btext%3D%22Unconditional%22>;index=1.1
History-info: <sip:+4944136111397@unknown.invalid;cause=380;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D486&Privacy=history>;index=1.1.1;mp=1.1
History-info: <sip:+4944136111398@unknown.invalid;cause=486;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.2;mp=1.1
History-info: <sip:+4944136111394@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D500&Privacy=history>;index=1.1.3;mp=1.1
History-info: <sip:+4944136111395@unknown.invalid;cause=404;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D500&Privacy=history>;index=1.1.4;mp=1.1
History-info: <sip:+494413900195@unknown.invalid;cause=404;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.5;mp=1.1
History-info: <sip:+4944139022692@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.6;mp=1.1
History-info: <sip:+494413900197@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.7;mp=1.1
History-info: <sip:+4944139022733@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.8;mp=1.1
History-info: <sip:+4944139063182@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.9;mp=1.1
History-info: <sip:+4944199869416@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.10;mp=1.1
History-info: <sip:+4944199869425@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Reason=SIP%3Bcause%3D487&Privacy=history>;index=1.1.11;mp=1.1
History-info: <sip:+4944199869427@unknown.invalid;cause=487;user=phone;target=sip:+49800376462401%40unknown.invalid?Privacy=history>;index=1.1.12;mp=1.1
Content-Length: 376

#ERROR:
W5 Header History-info : string too long, size=2204, max size=1024
W0 Apply matching rule HIH_del_REID : false

#Step reproduce:
Step 1: Get the ibcf binary version 7.2.60
Step 2: Get the basic call sipp scenarial
Step 3: Modify header initial INVITE the same as provided INVITE message
Step 4: Config HMR to delete HIH
Step 5: Make the basic call between ibcf
Step 6: Checking error log:
W5 Header History-info : string too long, size=2204, max size=1024
</example>"""
'''
    return related

In [ ]:
def checkStopPoint(state: State):
    if "NOK" in state["reviewResult"]:
        print("\nCheckPoint NOK\n")
        return "NOK"
    print("\nCheckPoint OK\n")
    return "OK"

def checkUserReview(state: State):
    if "NOK" in state["userReview"]:
        print("\nUser review NOK\n")
        return "NOK"
    print("\nUser review OK\n")
    return "OK"



In [ ]:
def User(state: State):
    print("\n*********************user**************************\n")
    result = interrupt(
        # Interrupt information to surface to the client.
        # Can be any JSON serializable value.
        {
            "task": "Review the output from the LLM and make any necessary edits.",
            "bugDescription": state["bugDescription"]
        }
    )
    print(result)
    #print(result["edited_text"])
    # Update the state with the edited text
    return {
        "userReview": result 
    }

In [ ]:
def BugAnalysisAgent(state: State):
    print("\n*********************BugAnalysisAgent**************************\n")
    #TODO: Get the bug information from the bug tracker with the bugID
    #Step:
    #1. Ask user if they want to input extra information to agent => If yes, store them in userData State
    #2. Get all raw data from redmine, then take the needed information to identify what is the problem  => Store to bugAnalized
    #3. Store the description to bugDescription
    information = "Memory keep inceased after timer_cell is fired"

    msg = printSteam(llm1, f"Analyse the bug with the following information: {information}")
    return {"bugAnalized": msg, "bugDescription": information}

In [ ]:
def RetrievalAgent(state: State):
    print("\n*********************RetrievalAgent**************************\n")
    bugDescription = state["bugDescription"]
    print("INPUT: \n", bugDescription)

    #TODO: Get the related historical data from vectorDB with the bug description
    #need input from the BugAnalysisAgent

    #Call the retrieve function from the vectorDB
    msg = getRelatedBugs(bugDescription)
    print("OUTPUT:\n", msg)
    return {"historicalData": msg}

In [ ]:
def DecisionAgent(state: State, retry=0):
    print("\n*********************DecisionAgent**************************\n")
    
    #The brain of agentic reproducer
    #Take input from the bugAnalyser and RetrievalAgent then suggest the reproduction steps
    req_info = ["bugDescription", "historicalData"]

    for req in req_info:
        if req not in state:
            print("Missing INPUT!!!\n")
            return {"reproduceSuggestion": "NOK"}

    bugDescription = state["bugDescription"]
    historicalData = state["historicalData"]
    if retry:
        failReproduce = state["failReproduce"]

    print("INPUT:\n", bugDescription, historicalData)
    #historicalData is a few shot example for LLM to predict the reproduce steps
    #historicalData format: "Problem: Reproduce steps"
    intruction = f"You are a TestAgent and you need to reproduce the problem with the following information:\n"

    #few shot prompt
    prompt = f"Exmaple: {historicalData}\n EndofExample. \nStep to reproduce the problem with the following information: {bugDescription}"

    if retry:
        prompt = f"Reproduce Exmaple: {historicalData}\nEndofExample\nAttempting to reproduce the failure:{failReproduce}\nEnd of trying\nWith all above information give the step to reproduce the problem with the following information: {bugDescription}"

    msg = printSteam(llm1, prompt)

    print(f"OUTPUT:\n{msg} ENDOF DecisionAgent \n")
    return {"reproduceSuggestion": msg}


In [ ]:
# from langchain_core.tools import tool
# from langgraph.graph import MessagesState
# from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage

# llm2 = OllamaLLM(base_urk="http://localhost:11434", model = "llama3.2:3b", streaming=True)

# # Define tools
# @tool
# def multiply(a: int, b: int) -> int:
#     """Multiply a and b.

#     Args:
#         a: first int
#         b: second int
#     """
#     return a * b

# # Augment the LLM with tools
# tools = [multiply]
# tools_by_name = {tool.name: tool for tool in tools}
# llm_with_tools = llm2.bind_tools(tools)

# def llm_call(state: MessagesState):
#     """LLM decides whether to call a tool or not"""
#     return {
#         "messages": [
#             llm_with_tools.invoke(
#                 [
#                     SystemMessage(
#                         content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
#                     )
#                 ]
#                 + state["messages"]
#             )
#         ]
#     }

In [ ]:
def TestAgent(state: State):
    print("\n*********************TestAgent**************************\n")
    #TODO: Trigger function call to reproduce the problem
    if "reproduceSuggestion" not in state:

        print("OUTPUT:\n This test is NOK END of TestAgent\n")
        return {"testResult": "The test is NOK"}

    reproduceSuggestion = state["reproduceSuggestion"]
    print("INPUT:\n", reproduceSuggestion)

    if "testResult" in state:
        testResult = "The test is OK"
    else:
        testResult = "The test is NOK"
    print(f"OUTPUT:\n{testResult} ENDOF TestAgent\n")
    return {"testResult": testResult}

In [ ]:
def ReviewAgent(state: State):
    print("\n*********************ReviewAgent**************************\n")
    testResult = state["testResult"]
    print("INPUT: \n", testResult)

    msg = printSteam(llm1, f"Review the test result is OK or NOK, only print answer: {testResult}")
    print(f"OUTPUT:\n{msg} ENDOF ReviewAgent\n")
    return {"reviewResult": msg}

In [ ]:
workflow = StateGraph(State)

# Add nodes
workflow.add_node("BugAnalysisAgent", BugAnalysisAgent)
workflow.add_node("RetrievalAgent", RetrievalAgent)
workflow.add_node("DecisionAgent", DecisionAgent)
workflow.add_node("TestAgent", TestAgent)
workflow.add_node("ReviewAgent", ReviewAgent)
workflow.add_node("User", User)

# Add edges to connect nodes
workflow.add_edge(START, "BugAnalysisAgent")
# workflow.add_conditional_edges(
#     "BugAnalysisAgent", checkIDValid, {"Fail": "improve_joke", "Pass": END}
# )

workflow.add_edge("BugAnalysisAgent", "RetrievalAgent") 
workflow.add_edge("RetrievalAgent", "DecisionAgent")
workflow.add_edge("DecisionAgent", "TestAgent")
workflow.add_edge("TestAgent", "ReviewAgent")

workflow.add_conditional_edges(
    "ReviewAgent", checkStopPoint, {"NOK": "DecisionAgent", "OK": "User"}
)

workflow.add_conditional_edges(
    "User", checkUserReview, {"NOK": "DecisionAgent", "OK": END}
)
# Compile
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

# Show workflow
display(Image(graph.get_graph().draw_mermaid_png()))

# Invoke
thread_config = {"configurable": {"thread_id": "1"}}
state = graph.invoke({"bugID": 12345}, thread_config)

#value_from_human = "This is the testing for user input"
value_from_human = str(input())
# Resume the graph with the human's input
graph.invoke(Command(resume=value_from_human), config=thread_config)

human_review = str(input())

graph.invoke(Command(resume=human_review), config=thread_config)